In [1]:
from skimage import filters,io,morphology
from matplotlib import pyplot as plt
import numpy as np
import cv2
import ipywidgets as widgets
import IPython.core.display
from IPython.core.display import display, HTML
from PIL import Image, ImageOps
import io

In [3]:
def prepareBinaryImg(image):
    minPixel = np.amin(image)
    maxPixel = np.amax(image)
    
    middleValue = minPixel + (maxPixel-minPixel)/2
    binaryImg = [[0 for j in range(len(image[0]))] for i in range(len(image))]
    
    for i in range(len(image)):
        for j in range(len(image[i])):
            if image[i][j] > middleValue:
                binaryImg[i][j] = 1
            else:
                binaryImg[i][j] = 0
                
    return binaryImg

In [15]:
def getBlackWhiteContour(binaryImg):
    erosion = morphology.erosion(np.array(binaryImg))
    erosion = morphology.erosion(erosion)
    dilation = morphology.dilation(np.array(binaryImg))
    dilation = morphology.dilation(dilation)
    
    borderOutside = dilation - binaryImg
    borderInside = binaryImg - erosion
    contour = borderOutside + borderInside
    return contour
    

In [12]:
def findContours(contourImg, imgColor):
    contours, hierarchy = cv2.findContours(contourImg, cv2.RETR_FLOODFILL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(imgColor, contours, -1, (0,255,0), 3)
    return contours, imgColor

In [16]:
def prepareUpload():
    imageUpload = widgets.FileUpload(accept='image/*', multiple=False)
    sliderBox = widgets.HBox([widgets.VBox([widgets.Label('Upload image')]), 
             widgets.VBox([imageUpload])])
    display(sliderBox)
    processButton = widgets.Button(description='Start')
    output = widgets.Output()
    output2 = widgets.Output()
    display(processButton, output, output2)
    
    def onProcessButton(b):
        with output:
            for uploaded_filename in imageUpload.value:
                content = imageUpload.value[uploaded_filename]['content']
                image = Image.open(io.BytesIO(content))
                image_gray = ImageOps.grayscale(image)
                image = np.asarray(image)
                image_gray = np.asarray(image_gray)
                image_oryg = image.copy()
                
                kernel = np.ones((3,3),np.float32)/9
                image_gray = cv2.filter2D(image_gray,-1,kernel)
                
                binaryImg = prepareBinaryImg(image_gray)
                contourImg = getBlackWhiteContour(binaryImg)
                contours, imgColor = findContours(contourImg, image)
                
                fig=plt.figure(figsize=(17,50))
                fig.add_subplot(1, 3, 1)
                plt.imshow(image_oryg, cmap = 'gist_gray')
                fig.add_subplot(1, 3, 2)
                plt.imshow(contourImg, cmap = 'gist_gray')
                fig.add_subplot(1, 3, 3)
                plt.imshow(imgColor, cmap = 'gist_gray')
                
                plt.show()
    processButton.on_click(onProcessButton)


In [17]:
prepareUpload()


Button(description='Start', style=ButtonStyle())

Output()

Output()